In [1]:
import datetime
import time
import subprocess
import sys
import pandas as pd, numpy as np
import os; os.chdir('..')

In [2]:
threshold = .015
step_size = .02
iterations = 50

#LUZ 69, 70, 72
target_luz = pd.read_csv('.\\data\\calibration\\target_luz.csv').values.flatten()
du_target = 0.01  #Proportion of DU growth over the calibration period captured by these LUZs

In [3]:
for i in range(iterations):
    print 'ITERATION %s' % (i + 1)
    start_time = datetime.datetime.now()
    print 'Starting iteration at: %s' % start_time
    
    # Simulation run
    subprocess.check_call([sys.executable, 'c://sandag//sandag_urbansim//calib_luz_simulation.py'])
    
    # Read LUZ result
    time.sleep(1)
    proportion_du = pd.read_csv('.\\data\\calibration\\luz_du_simulated.csv').values[0][0]
    
    # Get existing shifter
    shift = pd.read_csv('.\\data\\calibration\\luz_du_shifter.csv').values[0][0]
    
    target = du_target
    simulated = proportion_du
    print '    Target proportion is %s' % target
    print '    Simulated proportion is %s' % simulated
            
    difference = target - simulated

    if abs(difference) > threshold:
        if difference > 0:
            print '        Action: shift up'
            new_shift = shift + step_size
        else:
            print '        Action: shift down'
            new_shift = shift - step_size
        print '        New shift value %s' % new_shift
    else:
        print '        No action.  Done.'
        break
    
    # Write new shifter value out
    pd.DataFrame([new_shift]).to_csv('.\\data\\calibration\\luz_du_shifter.csv', index = False)
    
    end_time = datetime.datetime.now()
    print 'Iteration elapsed time: %s' % (end_time - start_time)
    time.sleep(1)

ITERATION 1
Starting iteration at: 2015-05-28 11:33:36.821000
    Target proportion is 0.01
    Simulated proportion is 0.032234058387
        Action: shift down
        New shift value 0.78
Iteration elapsed time: 0:03:09.346000
ITERATION 2
Starting iteration at: 2015-05-28 11:36:47.168000
    Target proportion is 0.01
    Simulated proportion is 0.0227596319357
        No action.  Done.
